## 1) Obtain genes

In [ ]:
# execute this cell to authorize the notebook
from google.colab import drive
drive.mount("./drive", force_remount=True)

Mounted at ./drive


In [ ]:
# execute this cell to bind the file in drive to the code
from os.path import join
import pandas as pd
import numpy as np

path_prefix = "./drive/My Drive/PURE-DNA_damage_repair/"

In [ ]:
# getting df of list of all genes

fname_in_ner = "maf_analysis_all_repair/ncbi_all_gene_result_27122023.txt"

with open(join(path_prefix, fname_in_ner), "r") as f_in:
  df_in = pd.read_csv(f_in, delimiter='\t')           # our list of genes

df_all = df_in.dropna(axis="columns", how="all")       # keeping only filled columns
df_all = df_all.dropna(axis="rows", how="all")

df_all

,tax_id,Org_name,GeneID,CurrentID,Status,Symbol,Aliases,description,other_designations,map_location,chromosome,genomic_nucleotide_accession.version,start_position_on_the_genomic_accession,end_position_on_the_genomic_accession,orientation,exon_count,OMIM
0,9606,Homo sapiens,7157,0,live,TP53,"BCC7, BMFS5, LFS1, P53, TRP53",tumor protein p53,cellular tumor antigen p53|antigen NY-CO-13|mu...,17p13.1,17,NC_000017.11,7668421.0,7687490.0,minus,13.0,191170
1,9606,Homo sapiens,1956,0,live,EGFR,"ERBB, ERBB1, ERRP, HER1, NISBD2, PIG61, mENA",epidermal growth factor receptor,epidermal growth factor receptor|EGFR vIII|avi...,7p11.2,7,NC_000007.14,55019017.0,55211628.0,plus,32.0,131550
2,9606,Homo sapiens,348,0,live,APOE,"AD2, APO-E, ApoE4, LDLCQ5, LPG",apolipoprotein E,apolipoprotein E|apolipoprotein E3,19q13.32,19,NC_000019.10,44905796.0,44909393.0,plus,6.0,107741
3,9606,Homo sapiens,7124,0,live,TNF,"DIF-alpha, TNFA, TNFSF2, TNLG1F, TNF",tumor necrosis factor,"tumor necrosis factor|APC1 protein|TNF, macrop...",6p21.33,6,NC_000006.12,31575565.0,31578336.0,plus,4.0,191160
4,9606,Homo sapiens,7422,0,live,VEGFA,"L-VEGF, MVCD1, VEGF, VPF",vascular endothelial growth factor A,"vascular endothelial growth factor A, long for...",6p21.1,6,NC_000006.12,43770211.0,43786487.0,plus,9.0,192240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20622,9606,Homo sapiens,100996709,0,live,LOC100996709,NaN,ADP-ribosylation factor-like protein 17,ADP-ribosylation factor-like protein 17,NaN,17,NaN,NaN,NaN,NaN,NaN,NaN
20623,9606,Homo sapiens,100996701,0,live,LOC100996701,NaN,uncharacterized LOC100996701,uncharacterized protein LOC100996701,12q24.33,12,NC_000012.12,131663774.0,131664548.0,minus,1.0,NaN
20624,9606,Homo sapiens,100996696,0,live,LOC100996696,NaN,polyadenylate-binding protein 4-like,polyadenylate-binding protein 4-like,12q14.1,12,NC_000012.12,59812094.0,59812618.0,plus,1.0,NaN
20625,9606,Homo sapiens,100507221,0,live,LOC100507221,NaN,uncharacterized LOC100507221,uncharacterized protein LOC100507221,16p11.2,16,NC_000016.10,32264029.0,32280475.0,minus,5.0,NaN


## 2) NCBI Data Acquisition

#### a) Install NCBI API

In [ ]:
# use a virtual environment
!apt install python3.10-venv

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3.10-venv is already the newest version (3.10.12-1~22.04.3).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [ ]:
# create the virtual environment using the name .venv_datasets
!python -m venv .venv_datasets
!source .venv_datasets/bin/activate

In [ ]:
# install wheel
!pip install wheel

In [ ]:
# install the latest available datasets package
!pip install --upgrade ncbi-datasets-pylib

In [ ]:
# verify installation
!python -c 'import ncbi.datasets.openapi; print(ncbi.datasets.openapi.__version__)'

16.6.0


#### b) Get SwissProt ID of each gene

In [ ]:
from typing import List

from ncbi.datasets.openapi import ApiClient as DatasetsApiClient
from ncbi.datasets.openapi import ApiException as DatasetsApiException
from ncbi.datasets import GeneApi as DatasetsGeneApi
from ncbi.datasets.openapi.models import V1GeneMatch

In [ ]:
def return_swissprot_information(gene_record: V1GeneMatch):
  # print any errors that occured while executing the above query, e.g. invalid identifier
  if gene_record.warnings:
    with open(join(path_prefix, "maf_analysis_all_repair/warnings.txt"), "a+") as f:
      f.write(str(gene_record.warnings))
      f.write("\n\n")
      return "NotFound"
  if gene_record.errors:
    with open(join(path_prefix, "maf_analysis_all_repair/errors.txt"), "a+") as f:
      f.write(str(gene_record.errors))
      f.write("\n\n")
      return "NotFound"

  # print gene metadata fields
  if gene_record.gene:
    gene_dictionary = gene_record.gene.to_dict()
    if "swiss_prot_accessions" not in gene_dictionary.keys():
      with open(join(path_prefix, "maf_analysis_all_repair/genes_without_swissprot_id_found.txt"), "a+") as f:
        f.write(str(gene_dictionary))
        f.write("\n\n")
      return "NotFound"
    else:
      return gene_dictionary['swiss_prot_accessions']

In [ ]:
# get transcripts for all repair genes

# provide gene ids as a list of integers
gene_ids = list(df_all['GeneID'])

# create small chunks for api to work
chunk_size = 100
chunks = [gene_ids[i:i + chunk_size] for i in range(0, len(gene_ids), chunk_size)]

for chunk in chunks:
  with DatasetsApiClient() as api_client:
    gene_api = DatasetsGeneApi(api_client)

    try:
      # retrieve gene metadata for the list of gene ids
      gene_reply = gene_api.gene_metadata_by_id(chunk)
      for gene in gene_reply.genes:
        # get swissprot id list
        swissprot = return_swissprot_information(gene)
        # write into df_all
        if swissprot == "NotFound":
          df_all.loc[df_all['GeneID'] == int(gene.query[0]), 'swissprot_id'] = swissprot
        else:
          all_ids = str(swissprot[0])
          for id in swissprot[1:]:
            all_ids += ", " + str(id)
          df_all.loc[df_all['GeneID'] == int(gene.query[0]), 'swissprot_id'] = all_ids

    except DatasetsApiException as e:
      print(f"Exception when calling GeneApi: {e}\n")

df_all

,tax_id,Org_name,GeneID,CurrentID,Status,Symbol,Aliases,description,other_designations,map_location,chromosome,genomic_nucleotide_accession.version,start_position_on_the_genomic_accession,end_position_on_the_genomic_accession,orientation,exon_count,OMIM,swissprot_id
0,9606,Homo sapiens,7157,0,live,TP53,"BCC7, BMFS5, LFS1, P53, TRP53",tumor protein p53,cellular tumor antigen p53|antigen NY-CO-13|mu...,17p13.1,17,NC_000017.11,7668421.0,7687490.0,minus,13.0,191170,P04637
1,9606,Homo sapiens,1956,0,live,EGFR,"ERBB, ERBB1, ERRP, HER1, NISBD2, PIG61, mENA",epidermal growth factor receptor,epidermal growth factor receptor|EGFR vIII|avi...,7p11.2,7,NC_000007.14,55019017.0,55211628.0,plus,32.0,131550,P00533
2,9606,Homo sapiens,348,0,live,APOE,"AD2, APO-E, ApoE4, LDLCQ5, LPG",apolipoprotein E,apolipoprotein E|apolipoprotein E3,19q13.32,19,NC_000019.10,44905796.0,44909393.0,plus,6.0,107741,P02649
3,9606,Homo sapiens,7124,0,live,TNF,"DIF-alpha, TNFA, TNFSF2, TNLG1F, TNF",tumor necrosis factor,"tumor necrosis factor|APC1 protein|TNF, macrop...",6p21.33,6,NC_000006.12,31575565.0,31578336.0,plus,4.0,191160,P01375
4,9606,Homo sapiens,7422,0,live,VEGFA,"L-VEGF, MVCD1, VEGF, VPF",vascular endothelial growth factor A,"vascular endothelial growth factor A, long for...",6p21.1,6,NC_000006.12,43770211.0,43786487.0,plus,9.0,192240,P15692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20622,9606,Homo sapiens,100996709,0,live,LOC100996709,NaN,ADP-ribosylation factor-like protein 17,ADP-ribosylation factor-like protein 17,NaN,17,NaN,NaN,NaN,NaN,NaN,NaN,NotFound
20623,9606,Homo sapiens,100996701,0,live,LOC100996701,NaN,uncharacterized LOC100996701,uncharacterized protein LOC100996701,12q24.33,12,NC_000012.12,131663774.0,131664548.0,minus,1.0,NaN,NotFound
20624,9606,Homo sapiens,100996696,0,live,LOC100996696,NaN,polyadenylate-binding protein 4-like,polyadenylate-binding protein 4-like,12q14.1,12,NC_000012.12,59812094.0,59812618.0,plus,1.0,NaN,NotFound
20625,9606,Homo sapiens,100507221,0,live,LOC100507221,NaN,uncharacterized LOC100507221,uncharacterized protein LOC100507221,16p11.2,16,NC_000016.10,32264029.0,32280475.0,minus,5.0,NaN,NotFound


In [ ]:
# save into a file
df_all.to_csv(join(path_prefix, "maf_analysis_all_repair/df_all_w_most_swissprot_ids.txt"), sep="\t")

#### c) Find the canonical swissprot ID for those with multiple IDs (manual check)

In [ ]:
print(df_all[df_all["swissprot_id"].str.contains(", ")].shape)
df_all[df_all["swissprot_id"].str.contains(", ")]

(55, 18)


,tax_id,Org_name,GeneID,CurrentID,Status,Symbol,Aliases,description,other_designations,map_location,chromosome,genomic_nucleotide_accession.version,start_position_on_the_genomic_accession,end_position_on_the_genomic_accession,orientation,exon_count,OMIM,swissprot_id
32,9606,Homo sapiens,1029,0,live,CDKN2A,"ARF, CAI2, CDK4I, CDKN2, CMM2, INK4, INK4A, ML...",cyclin dependent kinase inhibitor 2A,cyclin-dependent kinase inhibitor 2A|CDK4 inhi...,9p21.3,9,NC_000009.12,21967752.0,21995324.0,minus,10.0,600160,"P42771, Q8N726"
134,9606,Homo sapiens,5621,0,live,PRNP,"ASCR, AltPrP, CD230, CJD, GSS, KURU, PRIP, PrP...",prion protein (Kanno blood group),major prion protein|alternative prion protein|...,20p13,20,NC_000020.11,4686456.0,4701588.0,plus,2.0,176640,"F7VJQ1, P04156"
292,9606,Homo sapiens,2778,0,live,GNAS,"AHO, C20orf451, GPSA, GSA, GSP, NESP, PITA3, P...",GNAS complex locus,protein ALEX|protein GNAS|protein SCG6 (secret...,20q13.32,20,NC_000020.11,58839748.0,58911192.0,plus,22.0,139320,"O95467, P63092, P84996, Q5JWF2"
302,9606,Homo sapiens,796,0,live,CALCA,"CALC1, CGRP, CGRP-I, CGRP-alpha, CGRP1, CT, KC...",calcitonin related polypeptide alpha,calcitonin|calcitonin gene-related peptide 1|a...,11p15.2,11,NC_000011.10,14966668.0,14972351.0,minus,7.0,114130,"P01258, P06881"
967,9606,Homo sapiens,1649,0,live,DDIT3,"AltDDIT3, C/EBPzeta, CEBPZ, CHOP, CHOP-10, CHO...",DNA damage inducible transcript 3,DNA damage-inducible transcript 3 protein|C/EB...,12q13.3,12,NC_000012.12,57516588.0,57520517.0,minus,4.0,126337,"P0DPQ6, P35638"
1199,9606,Homo sapiens,2074,0,live,ERCC6,"ARMD5, CKN2, COFS, COFS1, CSB, CSB-PGBD3, POF1...","ERCC excision repair 6, chromatin remodeling f...",DNA excision repair protein ERCC-6|ERCC6-PGBD3...,10q11.23,10,NC_000010.11,49434881.0,49539538.0,minus,23.0,609413,"P0DP91, Q03468"
1217,9606,Homo sapiens,5170,0,live,PDPK1,"PDK1, PDPK2, PDPK2P, PRO0461",3-phosphoinositide dependent protein kinase 1,3-phosphoinositide-dependent protein kinase 1|...,16p13.3,16,NC_000016.10,2538021.0,2603188.0,plus,17.0,605213,"O15530, Q6A1A2"
1318,9606,Homo sapiens,27113,0,live,BBC3,"JFY-1, JFY1, PUMA",BCL2 binding component 3,bcl-2-binding component 3|p53 up-regulated mod...,19q13.32,19,NC_000019.10,47220824.0,47232860.0,minus,8.0,605854,"Q96PG8, Q9BXH1"
1705,9606,Homo sapiens,6241,0,live,RRM2,"C2orf48, R2, RR2, RR2M",ribonucleotide reductase regulatory subunit M2,ribonucleoside-diphosphate reductase subunit M...,2p25.1,2,NC_000002.12,10122568.0,10211010.0,plus,14.0,180390,"P31350, Q96LS8"
1757,9606,Homo sapiens,9378,0,live,NRXN1,"Hs.22998, PTHSL2, SCZD17",neurexin 1,neurexin-1|neurexin I,2p16.3,2,NC_000002.12,49918503.0,51032132.0,minus,27.0,600565,"P58400, Q9ULB1"


In [ ]:
df_all.at[32, 'swissprot_id'] = "P42771"
df_all.at[134, 'swissprot_id'] = "P04156"
df_all.at[292, 'swissprot_id'] = "Q5JWF2"
df_all.at[302, 'swissprot_id'] = "P06881"
df_all.at[967, 'swissprot_id'] = "P35638"
df_all.at[1199, 'swissprot_id'] = "Q03468"
df_all.at[1217, 'swissprot_id'] = "O15530"
df_all.at[1318, 'swissprot_id'] = "Q9BXH1"
df_all.at[1705, 'swissprot_id'] = "P31350"
df_all.at[1757, 'swissprot_id'] = "Q9ULB1"
df_all.at[2179, 'swissprot_id'] = "P0DTU4"
df_all.at[2206, 'swissprot_id'] = "Q14160"
df_all.at[2372, 'swissprot_id'] = "P39880"
df_all.at[2440, 'swissprot_id'] = "P24723"
df_all.at[2655, 'swissprot_id'] = "P42166"
df_all.at[2758, 'swissprot_id'] = "P0DSE1"
df_all.at[2820, 'swissprot_id'] = "O95278"
df_all.at[2985, 'swissprot_id'] = "P98175"
df_all.at[4129, 'swissprot_id'] = "Q9UPN3"
df_all.at[4221, 'swissprot_id'] = "Q13765"
df_all.at[4511, 'swissprot_id'] = "Q8N2E6"
df_all.at[5483, 'swissprot_id'] = "Q9Y4C0"
df_all.at[5552, 'swissprot_id'] = "Q92614"
df_all.at[6851, 'swissprot_id'] = "Q70YC5"
df_all.at[7095, 'swissprot_id'] = "P0CAP2"
df_all.at[7562, 'swissprot_id'] = "P0DPB6"
df_all.at[8137, 'swissprot_id'] = "Q8NFQ8"
df_all.at[8609, 'swissprot_id'] = "Q9P2S2"
df_all.at[8949, 'swissprot_id'] = "Q0VDF9"
df_all.at[9023, 'swissprot_id'] = "O96007"
df_all.at[10469, 'swissprot_id'] = "Q15155"
df_all.at[10745, 'swissprot_id'] = "O43687"
df_all.at[11534, 'swissprot_id'] = "O00241"
df_all.at[11661, 'swissprot_id'] = "P0DMN0"
df_all.at[11985, 'swissprot_id'] = "Q5R372"
df_all.at[12355, 'swissprot_id'] = "Q86VQ6"
df_all.at[13174, 'swissprot_id'] = "Q6PID8"
df_all.at[13722, 'swissprot_id'] = "Q9UPX0"
df_all.at[14018, 'swissprot_id'] = "Q15195"
df_all.at[14176, 'swissprot_id'] = "A0A096LNW5"
df_all.at[14640, 'swissprot_id'] = "Q96CS4"
df_all.at[14706, 'swissprot_id'] = "A8MQT2"
df_all.at[15013, 'swissprot_id'] = "Q6PI77"
df_all.at[15411, 'swissprot_id'] = "Q9BQS2"
df_all.at[15534, 'swissprot_id'] = "Q96G79"
df_all.at[16778, 'swissprot_id'] = "P0C7T4"
df_all.at[16845, 'swissprot_id'] = "Q13066"
df_all.at[16846, 'swissprot_id'] = "A6NHX0"
df_all.at[16982, 'swissprot_id'] = "Q5HYW2"
df_all.at[17760, 'swissprot_id'] = "Q9ULG3"
df_all.at[18165, 'swissprot_id'] = "Q9Y536"
df_all.at[19160, 'swissprot_id'] = "P0CG35"
df_all.at[19219, 'swissprot_id'] = "Q96N58"
df_all.at[19297, 'swissprot_id'] = "P0DW12"
df_all.at[19523, 'swissprot_id'] = "Q3ZM63"

In [ ]:
# save into a file
df_all.to_csv(join(path_prefix, "maf_analysis_all_repair/df_all_w_most_swissprot_ids_unique.txt"), sep="\t")

## 3) UniProt Data Acquisition

#### a) Install UniProt API

In [ ]:
!pip install bioservices
from bioservices import UniProt
from io import StringIO

#### b) Get protein length of all genes

In [ ]:
# initialise uniprot
u = UniProt()

for index, row in df_all[df_all["swissprot_id"] != "NotFound"].iterrows():
  # get uniprot id
  uniprot_id = row['swissprot_id']

  # get gene result
  result = u.search(uniprot_id)
  result_df = pd.read_csv(StringIO(result), sep='\t')

  # check if there is only one entry
  if result_df.shape[0] != 1:
    result_df = result_df[result_df['Reviewed']=="reviewed"]

  # write the length into df_all
  df_all.loc[index, "protein_length"] = int(result_df.iloc[0]["Length"])

df_all

,tax_id,Org_name,GeneID,CurrentID,Status,Symbol,Aliases,description,other_designations,map_location,chromosome,genomic_nucleotide_accession.version,start_position_on_the_genomic_accession,end_position_on_the_genomic_accession,orientation,exon_count,OMIM,swissprot_id,protein_length
0,9606,Homo sapiens,7157,0,live,TP53,"BCC7, BMFS5, LFS1, P53, TRP53",tumor protein p53,cellular tumor antigen p53|antigen NY-CO-13|mu...,17p13.1,17,NC_000017.11,7668421.0,7687490.0,minus,13.0,191170,P04637,393.0
1,9606,Homo sapiens,1956,0,live,EGFR,"ERBB, ERBB1, ERRP, HER1, NISBD2, PIG61, mENA",epidermal growth factor receptor,epidermal growth factor receptor|EGFR vIII|avi...,7p11.2,7,NC_000007.14,55019017.0,55211628.0,plus,32.0,131550,P00533,1210.0
2,9606,Homo sapiens,348,0,live,APOE,"AD2, APO-E, ApoE4, LDLCQ5, LPG",apolipoprotein E,apolipoprotein E|apolipoprotein E3,19q13.32,19,NC_000019.10,44905796.0,44909393.0,plus,6.0,107741,P02649,317.0
3,9606,Homo sapiens,7124,0,live,TNF,"DIF-alpha, TNFA, TNFSF2, TNLG1F, TNF",tumor necrosis factor,"tumor necrosis factor|APC1 protein|TNF, macrop...",6p21.33,6,NC_000006.12,31575565.0,31578336.0,plus,4.0,191160,P01375,233.0
4,9606,Homo sapiens,7422,0,live,VEGFA,"L-VEGF, MVCD1, VEGF, VPF",vascular endothelial growth factor A,"vascular endothelial growth factor A, long for...",6p21.1,6,NC_000006.12,43770211.0,43786487.0,plus,9.0,192240,P15692,395.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20622,9606,Homo sapiens,100996709,0,live,LOC100996709,NaN,ADP-ribosylation factor-like protein 17,ADP-ribosylation factor-like protein 17,NaN,17,NaN,NaN,NaN,NaN,NaN,NaN,NotFound,NaN
20623,9606,Homo sapiens,100996701,0,live,LOC100996701,NaN,uncharacterized LOC100996701,uncharacterized protein LOC100996701,12q24.33,12,NC_000012.12,131663774.0,131664548.0,minus,1.0,NaN,NotFound,NaN
20624,9606,Homo sapiens,100996696,0,live,LOC100996696,NaN,polyadenylate-binding protein 4-like,polyadenylate-binding protein 4-like,12q14.1,12,NC_000012.12,59812094.0,59812618.0,plus,1.0,NaN,NotFound,NaN
20625,9606,Homo sapiens,100507221,0,live,LOC100507221,NaN,uncharacterized LOC100507221,uncharacterized protein LOC100507221,16p11.2,16,NC_000016.10,32264029.0,32280475.0,minus,5.0,NaN,NotFound,NaN


In [ ]:
# save into a file
df_all.to_csv(join(path_prefix, "maf_analysis_all_repair/df_all_most_23032024.txt"), sep="\t")